In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Data processing:

In [53]:
import pandas as pd

df = pd.read_excel('data/History_2.xlsx')
df.head()

,Country,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009
0,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"21,069.00","20,372.00","16,795.00","20,273.00","23,155.00","22,053.00","27,190.00",NaN,NaN,NaN
1,Czechia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1555.86,1402.87,1527.92,1522.36,1561.29,1621.08,1625.33,1867.32,2375.41,2249.71
2,Denmark,5431.88,"6,511.00",6698.78,6061.73,6025.38,6364.14,"6,537.20",6274.61,5886.54,...,"10,330.30",12211.32,12919.72,14668.87,15994.84,18787.41,22790.95,27111.91,31652.36,25919.26
3,Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11810.62,11422.56,...,"9,081.00","9,427.00","9,465.00","9,184.00","9,233.00","8,692.00","8,909.00",NaN,NaN,NaN
4,Ireland,4326.26,"4,143.00",3721.72,3883.08,"4,643.30",5166.93,4917.89,4944.02,4890.29,...,"12,816.00","13,897.00","13,574.00","14,397.00","16,258.00","16,230.00",NaN,NaN,NaN,NaN


In [54]:
import math
Countries = {}

for index,data in df.iterrows():
    longest_chain = []
    current_longest = []
    start_year = 1985
    for i in range(1985,2010):
        value = data[str(i)]
        if type(value) == str:
            if  math.isnan(float(data[str(i)].replace(",",""))):
                start_year = start_year if len(current_longest) > len(longest_chain) else i - len(longest_chain) 
                current_longest = current_longest if len(current_longest) > len(longest_chain) else longest_chain
                longest_chain = []
            else:
                longest_chain.append(float(data[str(i)].replace(",","")))
        else:
            if  math.isnan(data[str(i)]):
                start_year = start_year if len(current_longest) > len(longest_chain) else i - len(longest_chain)
                current_longest = current_longest if len(current_longest) > len(longest_chain) else longest_chain
                longest_chain = []
            else:
                longest_chain.append(data[str(i)])
    start_year = start_year if len(current_longest) > len(longest_chain) else i - len(longest_chain) + 1            
    current_longest = current_longest if len(current_longest) > len(longest_chain) else longest_chain
    Countries[data["Country"]] = (current_longest,start_year)

### Model:

In [55]:
class LSTMModel(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

### Train the model:

### Predict the Price: